# GDP per capita

http://data.worldbank.org/indicator/NY.GDP.PCAP.CD  
http://api.worldbank.org/v2/en/indicator/ny.gdp.pcap.cd?downloadformat=excel

## Import the libraries

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn
import xlwings as xw

## Connect to the active Excel workbook

In [ ]:
wb = xw.Workbook.active()
wb

## Read the data

In [ ]:
data = xw.Range(1, 'B4').options(pd.DataFrame,
                                 expand='table').value

In [ ]:
meta = xw.Range(2, 'A1:D1').options(pd.DataFrame,
                                    expand='vertical',
                                    index=False).value.set_index('Country Code')

## Check it

In [ ]:
data.head()

In [ ]:
meta.head()

## Joining

In [ ]:
combined = pd.concat([meta, data],
                     axis=1,
                     join='inner')
combined

## Quick view

In [ ]:
xw.view(combined)

## Summary Statistics

In [ ]:
regions = combined.groupby('Region')['2014']
summary = regions.describe().unstack()
summary

## Plotting

In [ ]:
unstacked = combined.set_index([combined.index,
                                'Region'])['2014'].unstack().drop(np.nan, 1)
ax = unstacked.plot(kind='box',
                    rot=90,
                    ylim=(0, 100000),
                    return_type='axes')

## Write the results back to Excel

In [ ]:
wb.set_current()
xw.Sheet.add('2014')

In [ ]:
xw.Range('A1').value = 'GDP per capita 2014'
xw.Range('A2').value = summary.round(2)

In [ ]:
xw.Sheet('2014').autofit()

In [ ]:
fig = ax.get_figure()
xw.Plot(fig).show('GDP_PC_2014',
                  top=xw.Range('A12').top,
                  wkb=wb)